Detector de Lenguaje de Señas en tiempo real

Jesús Roberto Bollaín y Goytia Campos
Angel Yael Alcántara González

In [1]:
import pickle
import cv2
import mediapipe as mp
import numpy as np
import warnings

In [2]:
model_dict = pickle.load(open('./model.p', 'rb')) #modelo entrenado con Random Forest
model = model_dict['model']


#Setup de mediapipe
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

#solo permitimos que el modelo agarre 1 mano por razones de logica
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.6, max_num_hands=1)

letras = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'HOLA']
letrasDicc = {letra: letra for letra in letras}


#esto solo para quitar un warning molesto que no afecta
warnings.filterwarnings('ignore')

In [3]:
cap = cv2.VideoCapture(0) #creo que hay que cambiar el 0 por otro num dependiendo del dispositivo
letrasGuardadas = []

while True:
    dataNorm = []
    xLand = [] #landmarks
    yLand = []
    zLand = []

    ret, img = cap.read()
    img = cv2.flip(img, 1)
    height, width, _ = img.shape

    fuenteLetra = cv2.FONT_HERSHEY_COMPLEX

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    resultados = hands.process(img_rgb)
    if resultados.multi_hand_landmarks:
        for hand_landmarks in resultados.multi_hand_landmarks: #dibujo de las lineas de manos en video
            mp_drawing.draw_landmarks(
                img,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style()
            )

        for hand_landmarks in resultados.multi_hand_landmarks: #guardar los landmarks
            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                z = hand_landmarks.landmark[i].z

                xLand.append(x)
                yLand.append(y)
                zLand.append(z)

        #normalizacion, rendimiento no cambia mucho pero siento que es mejor con esta
        #es necesaria pq el modelo fue entrenado con datos normalizados 
        for i in range(len(hand_landmarks.landmark)):
            x = hand_landmarks.landmark[i].x
            y = hand_landmarks.landmark[i].y
            z = hand_landmarks.landmark[i].z
            dataNorm.append(x - min(xLand))
            dataNorm.append(y - min(yLand))
            dataNorm.append(z - min(zLand))
            #dataNorm.append(xLand)
            #dataNorm.append(yLand)
            #dataNorm.append(zLand)

        x1 = int(min(xLand) * width)
        y1 = int(min(yLand) * height)

        prediccion = model.predict([np.asarray(dataNorm)]) #prediccion con nuestro modelo segun los landmarks
        letraPredicha = letrasDicc[prediccion[0]] #letra que devuelve

        #letra actual con borde negro puesto horrible
        cv2.putText(img, letraPredicha, (x1+10,y1-25), fuenteLetra, 1.6, (0, 0, 0), 6, cv2.LINE_AA) 
        cv2.putText(img, letraPredicha, (x1+10,y1-25), fuenteLetra, 1.6, (255, 255, 255), 2, cv2.LINE_AA)

    key = cv2.waitKey(1)
    if key == 13: #Enter
        letrasGuardadas.append(letraPredicha) #guardamos letra al presionar enter
    elif key == 32: #Espacio
        letrasGuardadas.append(' ') #salto de linea
    elif key == 8: #Backspace o como se llame <-
        if letrasGuardadas:
            letrasGuardadas.pop() #borramos ultimo caracter
    elif key == 27: #Esc
        break #salimos del programa

    if len(letrasGuardadas) > 20: #si pasa de 20 letras, borramos todo
        letrasGuardadas = []

    palabraForm = ''.join(letrasGuardadas) #escribimos en la parte de abajo las letras guardadas
    cv2.putText(img, palabraForm, (10, height - 10), fuenteLetra, 1.4, (0, 0, 0), 6, cv2.LINE_AA)
    cv2.putText(img, palabraForm, (10, height - 10), fuenteLetra, 1.4, (255, 255, 255), 2, cv2.LINE_AA)
    
    cv2.imshow('img', img) #mostramos la imagen de la camara

cap.release()
cv2.destroyAllWindows()


: 